In [1]:
import json

In [2]:
with open('/Users/etashguha/Downloads/final_mle_dataset.json', 'r') as f:
    data = json.load(f)

In [3]:
print(len(data))

1017


In [5]:
data[0].keys()

dict_keys(['product_name', 'product_description', 'prospect_name', 'prospect_industry', 'prospect_title', 'email', 'accepted', 'critique', 'edited', 'email_embedding', 'edited_embedding'])

In [9]:
accepted = [obj for obj in data if obj.get('accepted') == 1]
rejected = [obj for obj in data if obj.get('accepted') == 0]

In [10]:
len(accepted)

524

In [11]:
len(rejected)

493

In [31]:
def separate_json_by_keys(json_data, keys):
    separated_jsons = {}
    for obj in json_data:
        key_values = tuple(obj[key] for key in keys)
        if key_values not in separated_jsons:
            separated_jsons[key_values] = []
        separated_jsons[key_values].append(obj)
    return separated_jsons
sort_key_list = ['prospect_industry']

sorted_jsons = separate_json_by_keys(data, sort_key_list)

In [91]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def remove_fillers(text):
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(token.lower(), pos='v') for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

[nltk_data] Downloading package punkt to /Users/etashguha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/etashguha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
import re
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.upper()
    return text

critical_words = []
changed_words = []
removed_words = []
for reject in rejected:
    critical_words.extend(preprocess_text(remove_fillers(reject["critique"])).split(" "))
    words_in_original_email = set(preprocess_text(remove_fillers(reject["email"])).split(" "))
    words_in_new_email = set(preprocess_text(remove_fillers(reject["edited"])).split(" "))
    changed_words.extend(list(words_in_new_email.difference(words_in_original_email)))
    removed_words.extend(list(words_in_original_email.difference(words_in_new_email)))  

In [126]:
from collections import Counter

def find_most_common(lst, n):
    counter = Counter(lst)
    most_common = counter.most_common(n)
    return most_common
# find_most_common(critical_words, 100) 
# rejected[104]["critique"]
find_most_common(changed_words, 100) 
# find_most_common(removed_words, 100) 

[('UNDERSTAND', 298),
 ('BEST', 255),
 ('REGARD', 207),
 ('DESIGN', 193),
 ('S', 188),
 ('FIND', 170),
 ('LET', 165),
 ('KNOW', 164),
 ('NEED', 162),
 ('LIKE', 156),
 ('LOVE', 149),
 ('DEAR', 149),
 ('INDUSTRY', 148),
 ('PROVIDE', 147),
 ('INTRODUCE', 140),
 ('DISCUSS', 139),
 ('WANT', 137),
 ('EMAIL', 133),
 ('HELP', 131),
 ('MAKE', 131),
 ('IMPORTANCE', 131),
 ('PLEASE', 127),
 ('SCHEDULE', 117),
 ('OFFER', 112),
 ('BELIEVE', 111),
 ('BENEFIT', 110),
 ('IMPORTANT', 109),
 ('LEARN', 106),
 ('WOULD', 103),
 ('ALSO', 99),
 ('CALL', 99),
 ('WORK', 97),
 ('FEATURE', 95),
 ('QUESTION', 93),
 ('INTEREST', 92),
 ('TIME', 92),
 ('D', 91),
 ('EXPERIENCE', 90),
 ('COME', 89),
 ('SPECIFIC', 89),
 ('UNIQUE', 84),
 ('MAY', 83),
 ('MEET', 83),
 ('SPECIFICALLY', 81),
 ('ENSURE', 76),
 ('REACH', 76),
 ('WELL', 76),
 ('LOOK', 74),
 ('KEEP', 68),
 ('RE', 68),
 ('SET', 68),
 ('US', 68),
 ('COULD', 67),
 ('CONSIDERATION', 64),
 ('HAPPY', 64),
 ('RANGE', 63),
 ('ANSWER', 62),
 ('CLIENTS', 62),
 ('TAKE', 6

In [99]:
import copy
accepted_names = []

def find_best_and_worst_categories(key):
    for accept in accepted:
        accepted_names.extend(remove_fillers(preprocess_text(accept[key])).split(" "))
    rejected_names = []
    for reject in rejected:
        rejected_names.extend(remove_fillers(preprocess_text(reject[key])).split(" "))
    good_names = copy.deepcopy(accepted_names)
    for bad_name in rejected_names:
        if bad_name in good_names:
            good_names.remove(bad_name)
    bad_names = copy.deepcopy(rejected_names)
    for good_name in accepted_names:
        if good_name in bad_names:
            bad_names.remove(good_name)
    print(find_most_common(good_names, 100))
    print(find_most_common(bad_names, 100))

find_best_and_worst_categories("prospect_industry")

[('entertainment', 19), ('fitness', 13), ('travel', 11), ('game', 10), ('education', 8), ('real', 7), ('estate', 7), ('beauty', 6), ('energy', 6), ('environmental', 4), ('arts', 4), ('transportation', 3), ('interior', 3), ('design', 3), ('architecture', 2), ('clean', 2), ('electrical', 2), ('art', 2), ('ecommerce', 2), ('agriculture', 2), ('law', 2), ('finance', 2), ('environment', 1), ('parent', 1), ('animal', 1), ('welfare', 1), ('', 1), ('dentistry', 1), ('care', 1), ('childcare', 1), ('tourism', 1), ('music', 1), ('landscape', 1), ('enforcement', 1), ('veterinary', 1), ('automotive', 1), ('renewable', 1), ('health', 1)]
[('healthcare', 19), ('hospitality', 12), ('sport', 9), ('beverage', 8), ('food', 8), ('business', 5), ('fashion', 5), ('service', 4), ('none', 4), ('technology', 4), ('pediatrics', 3), ('engineer', 2), ('event', 2), ('plan', 2), ('market', 2), ('security', 2), ('construction', 2), ('photography', 1), ('publish', 1), ('manufacture', 1), ('advertise', 1), ('appliance

# Analysis
We will now analyze how to improve the prompts of the GPT algorithm above. We will first examine the weaknesses of GPT.

## Weaknesses
We see that its weaknesses are across several industries: "healthcare," "hospitality," and "sport." Its best categories are "entertainment," "fitness," and "travel." These categories suggest that in industries where relatability and delicacy are essential traits, our GPT fails. In fact, among many of the critiques, a common thread was that the critics said that GPT was not personable. Looking at a word-by-word analysis, "empathy" and "personalization" were two of the most common complaints among the critiques.
Moreover, one of the most common differences between the original and edited emails was the greetings. GPT often said "hello" simply as a greeting, followed by very generic greetings. In the new emails, "dear," "best," and "regard" were far more common as salutations. These issues demonstrate that GPT needed to be more personable when writing emails. The new emails included "please" and other kinder words more often. Moreover, GPT often mentioned that it is a chatbot, as evidenced by the most commonly removed words. This phrase is very impersonal. 

Furthermore, GPT often fails when producing information about the industries. Two of the most common words in the critiques were "generic" and "relevant." Interestingly, these issues are relevant across many industries. This fact signifies that this is not an issue of GPT not knowing much about the industries but rather not knowing to give more details. Another critical insight is that one of the most common words in the critiques was "examples." GPT often needed to provide relevant examples that would have helped the pitch. 

Also, one of the most common words in the critiques was "prospect," and one of the most common words in the amended emails was "need." The amended emails often focused on addressing the project's specific needs, and GPT often needed to include information about the prospects. 

## Prompt Improvement Suggestions
I suggest several methods to improve the prompts.

### Few-Shot Prompting
To address the weakness above, we should give examples of poorly written and good emails according to the critical flaws noted above: personability in greetings and vocabulary, being too generic, giving examples, etc. This method is a common technique for improving the prompts. We should list the weaknesses above alongside examples of emails that fix and don't fix the fault to help GPT lean towards good emails. 

### Set Rules
We need to set some rules. We can include in the prompt explicit rules such as "say please more often" and "do not mention you are a chatbot." We can set these rules per the weaknesses above to weed out poor behaviors of GPT. 

### Generate Knowledge Prompting
We can ask GPT to predict how the prospect could use the product. Then, finally, we can tell GPT in a final promptly to include this information in the email. This addition to the prompting could help GPT to focus on more details of the prospect that genuinely benefit the prospect and be more effective at sales. 

### Simulus Prompting
We can ask GPT itself to rank several emails given the prompts. Then, using this reranker, we can generate several potential emails and output the one that GPT identifies that fixes the weaknesses above. This technique has been shown to reflect human annotations more reliably than zero-shot prompting, in general, more empirically. 

